In [39]:
import speech_recognition as sr
import pandas as pd
import numpy as np
import nltk

In [3]:
from gensim.models.keyedvectors import KeyedVectors
w2vec = KeyedVectors.load_word2vec_format("glove.6B.50d.txt", binary=False)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
data = pd.DataFrame(columns = ['Things to remember']) 

In [6]:
data

,Things to remember


In [7]:
df = pd.DataFrame(columns = ['Things to remember']) 

In [12]:
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Speak Anything :")
    audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        print("You said : {}".format(text))
    except:
        print("Sorry could not recognize what you said, please say again")
        text = input("Please write down you input ")

#data["Things to remember"] = [text] 
df["Things to remember"] = [text]
data = data.append(df)
data = data.reset_index()
data = data.drop("index",axis=1)

Speak Anything :
You said : I love to play cricket


In [13]:
data

,Things to remember
0,my name is daivastana
1,I was born in Odisha
2,I live in Hyderabad
3,my age is 25
4,I love to play cricket


In [19]:
from tqdm import tqdm
title_vec = np.zeros((data.shape[0],50))
for i in tqdm(range(0,data.shape[0])):
    words = data["Things to remember"].iloc[i].split(" ")
    words = [x.strip() for x in words]
    ind_word_vecs = [w2vec.word_vec(x) for x in words if x in w2vec.vocab]
    title_vec[i] = np.array(ind_word_vecs).mean(axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 4992.03it/s]


In [21]:
title_vec = np.nan_to_num(title_vec)

In [44]:
from nltk import word_tokenize

r = sr.Recognizer()
with sr.Microphone() as source:
    print("Speak Anything :")
    audio = r.listen(source)
    try:
        title1 = r.recognize_google(audio)
        print("You asked : {}".format(title1))
    except:
        print("Sorry could not recognize what you said, please say again")
        title1 = input("Please write down your input ")

#title1 = input("Please enter what you want to be reminded of. ")
title_vec_new = np.array([w2vec.word_vec(x) for x in word_tokenize(title1) if x in w2vec.vocab]).mean(axis=0)
title_vec_new = np.nan_to_num(title_vec_new)

b=[]
for i in range(0,title_vec.shape[0]):
    a = np.array([title_vec[i],title_vec_new])
    b.append(cosine_similarity(a))
    
e=[]
for i in range(0,title_vec.shape[0]):
    d = b[i][1][0]
    e.append(d)
    
c = e.index(max(e))
answer = data["Things to remember"][c]

forms = {"am" : "are", "are" : "am", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours', 'you' : 'I', 'your' : 'my', 'yours' : 'mine', 'was' : 'were'}
def translate(word):
  if word.lower() in forms: return forms[word.lower()]
  return word

result = ' '.join([translate(word) for word in nltk.wordpunct_tokenize(answer)])
print(result)

Speak Anything :
You asked : where was born
you were born in Odisha
